# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
import textdistance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

In [3]:
N = sum(vocab.values())
def P(word, N=N):
    return vocab[word] / N

In [4]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)
    return similarities.most_common(topn)
# возвр слова, которые ближе всего по расстоянию редактирования

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]
    return [(id2word[top], similarities[top]) for top in topn]
    # возвр слова, которые ближе всего по косинусному расстоянию

# объединяем обе функции:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4) # находим topn кандидатов по косинусному расстоянию
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    # из них выбираем наиболее близкое по расстоянию редактирования
    # Было:
    return closest

In [5]:
# Результат работы функции был такой:
get_closest_hybrid_match('водыца', X, vec)

[('воды', 0.6666666666666667), ('водыжу', 0.6666666666666667), ('своды', 0.5)]

In [6]:
# у нас получилось 3 претендента, 2 из которых имеют одинаковое расстояние редактирования:
# ('воды', 0.6666666666666667), ('водыжу', 0.6666666666666667)
# Проверим буквально, какое из них более вероятное:
print (P('воды'))
print (P('водыжу'))
print (max(P('воды'),P('водыжу')))

0.00013442531432343428
1.939759225446382e-07
0.00013442531432343428


In [7]:
# более вероятным оказалось слово "воды"
# Переопределим последнюю функцию с учетом выбора по вероятности:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4) # находим topn кандидатов по косинусному расстоянию
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    # из них выбираем наиболее близкое по расстоянию редактирования
    return max(closest, key=P)

In [8]:
get_closest_hybrid_match('водыца', X, vec)
# всё работает правильно

('воды', 0.6666666666666667)

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [9]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

In [10]:
def delete_letters(word_from_vocab):
    "Удаляем из слова по одной букве"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word_from_vocab[:i], word_from_vocab[i:])    for i in range(len(word_from_vocab) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

cursed_vocab = {} # генерим словарь где ключ - слово с удаленной буквой, значение - список из всех адекватных слов
# соответствующих этому удалению
for w in vocab:
    for i in delete_letters(w):
        if i in cursed_vocab:
            cursed_vocab[i].append(w)
        else:
            cursed_vocab[i] = [w]

In [11]:
def best_fit(word_from_input):
    "Берем на вход слово из инпута, генерим к нему удаления, выбираем к каждому из словаря соответствий самое частотное слово"
    words = delete_letters(word_from_input)
    # ищем совпадения в словаре удалений
    candidates = []
    for i in words:
        if i in cursed_vocab:
            candidates.append(max(cursed_vocab[i], key=P))
    if len(candidates) == 0:
        candidates.append(word_from_input)
    return max(candidates, key=P)

In [12]:
print (best_fit('медвеженок'))
print (best_fit('лошедб'))
# работает

медвежонок
лошедб


In [13]:
# проверяем, насколько хорош алгоритм
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2) # просто проверяем что длина одинаковая.
    
    return list(zip(tokens_1, tokens_2)) # zip = итератор, который объединяет элементы из нескольких ист-в данных (в кортеж?)

In [14]:
correct = 0 # правильных слов
total = 0 # всего слов

total_mistaken = 0 # 
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {} # словарь слов, которые мы уже подавали на исправление
# ключ - слово как оно было, значение = исправление
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i]) # в очередной раз соединили слова из двух словарей в пары
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], best_fit(pair[1]))
        # dict.get(key[, default])
        # Метод dict.get() возвращает значение для ключа key, если ключ находится в словаре
        # если ключ отсутствует то вернет значение default.
        cashed[pair[1]] = predicted # если исправленного слова нет в cashed, добавляем его
        # это мы прям все слова из двух корпусов проверяем
        
        if predicted == pair[0]:
            correct += 1 # если слово, которое получилось, совпало с исходным, добавляем в правильные +1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1 # считаем сколько изначально было правильных
            if pair[0] !=  predicted:
                correct_broken += 1 # считаем сколько иправилось с правильного на неправильный
        else:
            total_mistaken += 1 # всего было неправильных
            if pair[0] == predicted:
                mistaken_fixed += 1 # исправилось правильно
        
    if not i % 100:
        print(i) # просто следим чтоб работало?
        

0
100
200
300
400
500
600
700
800
900


In [15]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)
# вывод: алгоритм работает намного хуже, чем Норвиг из семинарской тетрадки.
# мой алгоритм правильно сгенерировал только 43% слов, исправился правильно всего 21% из слов с опечаткой
# из слов, которые были изначально правильные, алгоритм сломал больше половины

0.4346173086543272
0.21350931677018634
0.5326748593086023


## *3. Настройка гиперпараметров. (2 балла)

У метода из первого задания много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

In [3]:
# ваш кот тут